#### Login to Huggingface

In [ ]:
from huggingface_hub import login
login("YOUR_HUGGINGFACE_TOKEN")

#### Base Model

In [ ]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM

# MODEL_ID = "google/gemma-2b-it"

# tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_ID,
#     torch_dtype=torch.float16,
#     device_map="auto"
# )

#### Fine-tuned Model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

base = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2b-it",
    torch_dtype=torch.float16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("gemma-lora-webq-finetuned")

model = PeftModel.from_pretrained(base, "gemma-lora-webq-finetuned")
model.config.pad_token_id = tokenizer.pad_token_id


model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


#### Load Knowledge Graph

In [ ]:
import pandas as pd
df = pd.read_csv("knowledge_graph.csv")

filtered_df = df[["subject_label", "predicate_label", "object_label"]].copy()

filtered_df['subject_label'] = filtered_df['subject_label'].astype(str).str.strip()
filtered_df['predicate_label'] = filtered_df['predicate_label'].astype(str).str.strip()
filtered_df['object_label'] = filtered_df['object_label'].astype(str).str.strip()

filtered_df['subject_label_lower'] = filtered_df['subject_label'].str.lower()
filtered_df['predicate_label_lower'] = filtered_df['predicate_label'].str.lower()

#### Grab Unique Subjects & Predicts From Knowledge Graph

In [ ]:
subject_list = filtered_df['subject_label_lower'].unique()
predicate_list = filtered_df['predicate_label_lower'].unique()

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util

# Load a lightweight embedding model
embedder = SentenceTransformer("all-mpnet-base-v2")

filtered_df["triple_text"] = (
    filtered_df["subject_label"] + " | "
    + filtered_df["predicate_label"] + " | "
    + filtered_df["object_label"]
)

triples = filtered_df["triple_text"].tolist()

# Precompute embeddings once
triple_embeddings = embedder.encode(
    triples, convert_to_tensor=True, show_progress_bar=True, normalize_embeddings= True
)

#### Semantic search (Cosine Similarity)

In [ ]:
def get_semantic_facts(question, triples, triple_embeddings, top_k=3):
    
    q_emb = embedder.encode(question, convert_to_tensor=True, normalize_embeddings=True)
    cos_scores = util.cos_sim(q_emb, triple_embeddings)[0]
    top_idxs = torch.topk(cos_scores, k=top_k).indices.tolist()
    return [triples[i] for i in top_idxs]

def get_facts_for_question(question):
    try:
        return get_semantic_facts(question, triples, triple_embeddings, top_k=5)
    except Exception:
        return []

#### Extract Facts from Knowledge Graph

In [ ]:
# import difflib

# def extract_best_match(text, candidates, cutoff=0.6):
#     text = text.lower()

#     best_matches = difflib.get_close_matches(text, candidates, n=1, cutoff=cutoff)
#     if best_matches:
#         return best_matches[0]
    
#     matches = [c for c in candidates if c in text]
#     return max(matches, key=len) if matches else None

# def get_facts_for_question(question, df, subject_list, predicate_list):

#     matched_subject = extract_best_match(question, subject_list)
#     if matched_subject is None:
#         return []

#     subject_df = df[df["subject_label_lower"] == matched_subject]
#     if subject_df.empty:
#         return []

#     matched_predicate = extract_best_match(question, predicate_list)

#     if matched_predicate:

#         filtered_df = subject_df[
#             subject_df["predicate_label"].str.lower() == matched_predicate
#         ]
#         if not filtered_df.empty:
#             facts = [
#                 f"{row['subject_label']} — {row['predicate_label']} — {row['object_label']}"
#                 for _, row in filtered_df.iterrows()
#             ]
#             return facts

#     facts = [
#         f"{row['subject_label']} — {row['predicate_label']} — {row['object_label']}"
#         for _, row in subject_df.iterrows()
#     ]
#     return facts


#### Generate Answer using facts from Knowledge Graph

In [ ]:
def generate_answer(question, facts):
    formatted_facts = "\n".join(f"- {fact}" for fact in facts)
    prompt = f"""<bos>
[INST]
Using the facts below, answer the question with a short, direct answer.

Facts:
{formatted_facts}

Question: {question}
[/INST]
"""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=False
    )
    
    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if "[/INST]" in full_output:
        answer_only = full_output.split("[/INST]")[-1].strip()
    else:
        answer_only = full_output.strip()

    return answer_only


In [ ]:
# question = "what state does selena gomez?"

# facts = get_facts_for_question(question, filtered_df, subject_list, predicate_list)

# answer = generate_answer(question, facts)

# print(answer)

#### Questions from Webquestions Dataset (20%)

In [ ]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("stanfordnlp/web_questions", split="test")

full = load_dataset("stanfordnlp/web_questions")
full = full.filter(lambda ex: len(ex["answers"]) > 0)
split = full["train"].train_test_split(test_size=0.2, seed=42)
raw_train, raw_test = split["train"], split["test"]

questions_df = pd.DataFrame(raw_test)[['question', 'answers']]

#### Process One Question

In [ ]:
def process_question(question):
    # facts = get_facts_for_question(question, filtered_df, subject_list, predicate_list)
    facts = get_facts_for_question(question)
    
    if not facts:
        facts = ["No facts available."]
    return generate_answer(question, facts)

#### Answer all questions

In [ ]:
from tqdm import tqdm

tqdm.pandas()

questions_df['predicted_answer'] = questions_df['question'].progress_apply(process_question)

#### Prepare Data for Evaluation

In [ ]:
references_bleu = questions_df['answers'].tolist()
references_rouge = questions_df['answers'].apply(lambda x: ' / '.join(x)).tolist()

predictions = questions_df['predicted_answer'].tolist()

#### Evaluate Predictions

In [ ]:
import evaluate

bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

bleu_res = bleu.compute(predictions=predictions, references=references_bleu)
rouge_res = rouge.compute(predictions=predictions, references=references_rouge)

results_df = pd.DataFrame([{
    "bleu":      bleu_res["bleu"],
    "rouge1":    rouge_res["rouge1"],
    "rouge2":    rouge_res["rouge2"],
    "rougeL":    rouge_res["rougeL"],
    "rougeLsum": rouge_res["rougeLsum"],
}])

results_df.to_csv("evaluation_results.csv", index=False)